In [13]:
import pickle
import numpy as np
import datetime as dt
from tpot import TPOTClassifier
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import GroupKFold

`X_train` = array of training data, `y_train` = labels, `group` = protein cmplx group

In [14]:
tpot_dir = '../ppi_ml/tpot/'
timestamp = dt.datetime.now().strftime('%Y-%m-%d')
outfile = tpot_dir+'tpot_pipeline_'+timestamp
outfile

'../ppi_ml/tpot/tpot_pipeline_2023-02-15'

In [15]:
fmatfile = '../ppi_ml/data/featmats/featmat_labeled_traintest.pkl'
with open(fmatfile, 'rb') as handle:
    fmat = pickle.load(handle)

In [16]:
label_cols = ['ID', 'label', 'super_group']
data_cols = [c for c in fmat.columns.values.tolist() if c not in label_cols]

In [17]:
fmat[label_cols[0]]

0        ENOG502QR6E ENOG502QPKB
1            KOG0308 ENOG502QPKB
2            KOG0384 ENOG502QPKB
3            KOG0631 ENOG502QPKB
4            KOG0948 ENOG502QPKB
                  ...           
25236            KOG1894 KOG1597
25237            KOG1894 KOG1898
25238            KOG1894 KOG2691
25239            KOG1894 KOG2732
25240            KOG1894 KOG3400
Name: ID, Length: 25241, dtype: object

In [18]:
X = fmat[data_cols].to_numpy()
y = fmat[label_cols[1]].to_numpy()
groups = fmat[label_cols[2]].to_numpy()

In [33]:
gss = GroupShuffleSplit(n_splits=3, train_size=0.7, random_state=13)
gss.get_n_splits()
for i, (test_index, train_index) in enumerate(gss.split(X, y, groups)):
    print(f"Fold {i}:")
    print(f"\tTrain: index={train_index}, group={groups[train_index]}")
    print(f"\tTest:  index={test_index}, group={groups[test_index]}")
    print(f"\t # train PPIs = {len(X[train_index])}")
    print(f"\t # test PPIs = {len(X[test_index])}")

Fold 0:
	Train: index=[    2     6     7 ... 25238 25239 25240], group=[14  1 79 ...  1 47  1]
	Test:  index=[    0     1     3 ... 25223 25231 25237], group=[511 151  17 ...  23 339 962]
	 # train PPIs = 18339
	 # test PPIs = 6902
Fold 1:
	Train: index=[    0     4     6 ... 25236 25238 25240], group=[511  18   1 ...   1   1   1]
	Test:  index=[    1     2     3 ... 25235 25237 25239], group=[151  14  17 ...  74 962  47]
	 # train PPIs = 17148
	 # test PPIs = 8093
Fold 2:
	Train: index=[    2     3     6 ... 25236 25238 25240], group=[14 17  1 ...  1  1  1]
	Test:  index=[    0     1     4 ... 25235 25237 25239], group=[511 151  18 ...  74 962  47]
	 # train PPIs = 17764
	 # test PPIs = 7477


In [8]:
idx = gss.split(X, y, groups)
idx

<generator object BaseShuffleSplit.split at 0x7fc761731070>

In [10]:
for train_idx, test_idx in gss.split(X, y, groups):
    X_train = X[train_idx]
    y_train = y[train_idx]
    X_test = X[test_idx]
    y_test = y[test_idx]

In [11]:
pipeline_opt = TPOTClassifier()
pipeline_opt = TPOTClassifier(generations=10, population_size=20, cv=5,
                                    random_state=13, verbosity=2)

In [12]:
pipeline_opt.fit(X_train, y_train)
print(pipeline_opt.score(X_test, y_test))
pipeline_opt.export(outfile)

Optimization Progress:   0%|          | 0/220 [00:00<?, ?pipeline/s]



TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: ExtraTreesClassifier(input_matrix, bootstrap=True, criterion=gini, max_features=0.35000000000000003, min_samples_leaf=3, min_samples_split=14, n_estimators=100)


NameError: name 'X_test' is not defined